<a href="https://colab.research.google.com/github/ShinwooChoi/ESAA-OB/blob/main/11_03_ESAA_OB_%ED%95%84%EC%82%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

범위: <파이썬 머신러닝 완벽 가이드> 9장 p.625-647

### 07 행렬 분해를 이용한 잠재 요인 협업 필터링 실습



#### 개요
- 행렬 분해(Matrix Factorization)는 **잠재 요인(Latent Factor)** 을 이용한 협업 필터링 방법.  
- 사용자–아이템 평점 행렬을 두 개의 저차원 행렬로 분해하여 숨겨진 요인을 학습함.  
- 자주 사용되는 방법:
  - SVD (Singular Value Decomposition)
  - NMF (Non-negative Matrix Factorization)
  - 실제 시스템에서는 SGD(확률적 경사하강법)나 ALS(교대 최소제곱법)를 더 자주 사용.  
- 여기서는 **SGD 기반 행렬 분해**로 추천 시스템 구현.



#### 학습 원리
- 사용자–아이템 평점 행렬 R을 두 개의 행렬 P(사용자 행렬)와 Q(아이템 행렬)로 분해:
  \[
  R ≈ P × Q^T
  \]
- P: 사용자 × 잠재요인 행렬  
- Q: 아이템 × 잠재요인 행렬  
- 오차 최소화를 위한 손실함수:
  \[
  \sum (r_{ui} - p_u q_i^T)^2 + λ(||p_u||^2 + ||q_i||^2)
  \]
- λ: L2 정규화 항 (overfitting 방지)





### 08 파이썬 추천 시스템 패키지 – Surprise



#### Surprise 개요
- 추천 시스템 구축을 위한 **전용 파이썬 패키지**  
- 협업 필터링, SVD, NMF, KNN 등 다양한 알고리즘 지원  
- 학습–예측–평가를 위한 통합 API 제공  
- scikit-learn과 유사한 사용 방식 → 직관적 구현 가능  
- RMSE, MSE 등 **정확도 평가 기능 내장**



#### 설치
- pip install scikit-surprise  
- 또는 conda install -c conda-forge scikit-surprise  
- Windows 환경: Microsoft Build Tools 2015 이상 필요  



#### 주요 특징
- 사용자–아이템 기반 협업 필터링 간편 구현  
- `fit()`, `test()`, `predict()` 등 학습/예측 함수 제공  
- 내장 데이터(MovieLens) 및 커스텀 CSV/Pandas 데이터 모두 사용 가능  
- `accuracy` 모듈로 RMSE, MSE 계산  
- `GridSearchCV`로 하이퍼파라미터 튜닝 가능  



#### 기본 절차
1️⃣ Dataset 로드  
2️⃣ train_test_split으로 학습/테스트 데이터 분리  
3️⃣ 알고리즘(SVD 등) 선택 및 학습  
4️⃣ 예측 수행(test 또는 predict)  
5️⃣ RMSE 등으로 성능 평가  



#### 데이터 구조
- MovieLens 데이터셋 사용 (`ml-100k`, `ml-1m` 등)  
- 주요 컬럼: **userId, movieId, rating, timestamp**  
- Surprise의 Dataset 클래스 이용 → 내장 데이터 자동 로드 가능  
- `Dataset.load_builtin('ml-100k')` 으로 호출  



#### 주요 메서드 정의
- **fit()** : 학습 데이터 기반 모델 훈련  
- **test()** : 전체 테스트 세트에 대한 예측 수행  
- **predict()** : 개별 사용자와 아이템의 예측 평점 반환  
- **accuracy.rmse()** : 예측 평점과 실제 평점의 오차(RMSE) 계산  


#### Prediction 객체 구조
- uid : 사용자 ID  
- iid : 아이템(영화) ID  
- r_ui : 실제 평점  
- est : 예측 평점  
- details : 예측 성공 여부(‘was_impossible’ 포함)  
- 반환값 형태: 리스트 내 Prediction 객체  

#### 평가
- **RMSE**: Root Mean Squared Error  
  → 실제 평점과 예측 평점의 차이 제곱 평균의 제곱근  
- RMSE가 작을수록 예측 정확도가 높음  
- Surprise에서 `accuracy.rmse(predictions)` 으로 간단히 계산  


#### 주요 모듈
| 모듈명 | 설명 |
|--------|------|
| Dataset | 데이터 로드 및 분리 (내장 / 파일 / DataFrame) |
| Reader | 파일 형식(line_format, sep, rating_scale) 지정 |
| SVD, NMF, KNNBasic | 다양한 협업 필터링 알고리즘 제공 |
| accuracy | RMSE, MSE 등 성능 평가 |
| model_selection | train_test_split, GridSearchCV 제공 |

#### Dataset 클래스 주요 API
- **load_builtin(name)** : 내장 MovieLens 데이터 로드  
- **load_from_file(file_path, reader)** : CSV 등 외부 파일에서 로드  
- **load_from_df(df, reader)** : Pandas DataFrame에서 로드  

#### Reader 클래스 주요 파라미터
- **line_format** : 데이터 컬럼 순서 (‘user item rating timestamp’)  
- **sep** : 구분자 문자 (기본값 ‘,’)  
- **rating_scale** : 평점 최소~최대 범위 지정 (예: (0.5, 5.0))  



#### OS 파일 로드
- CSV 파일에서 데이터 읽을 때 Reader로 포맷 지정  
- 헤더 제거 후 ‘user, item, rating, timestamp’ 순서로 로드 필요  
- `Dataset.load_from_file()` 사용  



#### DataFrame 로드
- pandas DataFrame에서 바로 로딩 가능  
- `Dataset.load_from_df(ratings[['userId','movieId','rating']], reader)`  
- 형식만 일치하면 Surprise 내부에서 자동 변환  


#### SVD 알고리즘 특징
- 행렬 분해 기반 협업 필터링 알고리즘  
- 사용자와 아이템의 잠재 요인을 학습하여 평점 예측  
- 기본 파라미터: n_factors(요인 수), random_state, learning_rate 등  
- RMSE 약 0.86~0.95 수준으로 측정됨  



#### Surprise의 장점
- 간결하고 직관적인 코드 구조  
- 다양한 알고리즘 실험 가능  
- RMSE, MSE 등 평가 통합 제공  
- 실제 산업용 데이터셋(MovieLens 등) 바로 활용 가능  




#### 한계
- 딥러닝 기반 추천모델(딥 협업 필터링, AutoEncoder 등)은 미지원  
- 대규모 데이터셋 처리 속도는 다소 느림  
- 하이브리드 추천(콘텐츠 + 협업) 직접 구현은 불편  


### Surprise 추천 알고리즘 및 개인화 시스템


#### Surprise 추천 알고리즘 클래스
- **SVD** : 행렬 분해를 통한 잠재 요인 협업 필터링 알고리즘  
- **KNNBasic** : 최근접 이웃 기반 협업 필터링  
- **BaselineOnly** : 사용자·아이템 Bias를 고려한 베이스라인 알고리즘  
- 추가 지원 : SVD++, NMF, SlopeOne, Co-Clustering 등  



#### SVD의 예측 평점식
- 예측식: r̂ᵤᵢ = μ + bᵤ + bᵢ + qᵢᵗpᵤ  
  (평균 평점 + 사용자 편향 + 아이템 편향 + 잠재요인 내적)
- Regularization(정규화): 과적합 방지 위해 λ 적용  
  → Σ(rᵤᵢ - r̂ᵤᵢ)² + λ(bᵤ² + bᵢ² + ||qᵢ||² + ||pᵤ||²)


#### 주요 파라미터
| 파라미터 | 설명 |
|-----------|------|
| n_factors | 잠재 요인 개수(기본 100) |
| n_epochs | SGD 반복 횟수(기본 20) |
| biased | 편향 적용 여부(True 기본) |



#### 알고리즘 성능 비교
| 알고리즘 | RMSE | MAE | Time |
|-----------|------|------|------|
| SVD | 0.934 | 0.737 | 0:11 |
| SVD++ | 0.951 | 0.722 | 0:14 |
| NMF | 0.963 | 0.758 | 0:15 |
| KNN Baseline | 0.931 | 0.733 | 0:12 |
| BaselineOnly | 0.944 | 0.748 | 0:01 |

- SVD++ 성능 최고, 단 학습시간 김  
- SVD / KNNBaseline → 효율적 성능  
- Baseline : 편향 반영하여 예측 정확도 향상  



#### 베이스라인 평점 개념
- 개인의 평가 성향(편향)을 반영한 평점 예측 방식  
- 사용자 평균 평점이 전반적으로 높은 경우 → 모든 영화에 높은 점수 경향  
- 반대로 냉정한 사용자는 낮은 점수 경향 → 이를 보정하기 위해 Bias 적용  
- 구성요소: 전체 평균 평점 + 사용자 편향 + 아이템 편향  


#### 교차 검증 (Cross Validation)
- 모델의 일반화 성능을 평가하기 위한 검증 방법  
- `cross_validate()` 함수 사용  
- RMSE, MAE, Fit Time, Test Time 등 측정  
- Fold별 평균 및 표준편차 계산 → 모델 안정성 확인  



#### 하이퍼 파라미터 튜닝 (GridSearchCV)
- 파라미터 조합을 자동 탐색하여 최적 성능 찾기  
- Surprise의 `GridSearchCV` → scikit-learn 방식과 동일  
- 주요 튜닝 변수:  
  - n_epochs (반복 학습 횟수)  
  - n_factors (잠재 요인 수)  
- 예시 결과: n_epochs=20, n_factors=50 → RMSE 약 0.8769  



#### 개인화 영화 추천 시스템 개념
- 학습된 협업 필터링 모델(SVD 등)을 이용하여  
  특정 사용자에게 **아직 평가하지 않은 영화**를 예측하여 추천  
- 사용자별 취향을 반영한 맞춤형 추천 시스템 구현  


#### DatasetAutoFolds 클래스
- 전체 데이터셋을 학습용으로 사용할 때 이용  
- `build_full_trainset()` 메서드로 전체 데이터를 학습셋으로 변환  



#### 개인화 추천 절차
1️⃣ **전체 데이터 학습** : `build_full_trainset()`으로 모델 학습  
2️⃣ **특정 사용자 지정** : 예) userId=9  
3️⃣ **평가하지 않은 영화 확인**  
   → userId 9가 평점을 매기지 않은 movieId 추출  
4️⃣ **predict()로 평점 예측**  
   → 예상 평점(est) 출력  



#### 추천 대상 영화 추출
- 전체 영화 목록에서 사용자가 본 영화 제외  
- 남은 영화 = 추천 후보  
- 예시: 총 9742편 중 46편만 평가 → 추천 대상 9696편  



#### 추천 함수 구조
- `get_unseen_surprise()` : 사용자가 보지 않은 영화 리스트 반환  
- `recomm_movie_by_surprise()` :  
  - 추천 알고리즘 객체(algo), 사용자 ID, 미시청 영화 리스트 입력  
  - 각 영화별 예측 평점 계산 후 내림차순 정렬  
  - 상위 N개 영화 반환 (기본 10개)





### 09 요약
- Surprise는 **협업 필터링 기반 추천 시스템 구현에 특화된 파이썬 패키지**  
- 주요 알고리즘: SVD, KNN, Baseline, NMF 등  
- 교차 검증·튜닝·편향보정 기능까지 통합 제공  
- 실무용 개인화 추천 시스템 구현에 매우 유용  


In [1]:
import numpy as np

def get_rmse(R, P, Q, non_zeros):
    error = 0
    for i, j, r in non_zeros:
        pred = np.dot(P[i, :], Q[j, :].T)
        error += pow(r - pred, 2)
    rmse = np.sqrt(error / len(non_zeros))
    return rmse

In [2]:

def matrix_factorization(R, K, steps=200, learning_rate=0.01, r_lambda=0.01):
  num_users, num_items = R.shape

  # P와 Q 매트릭스의 크기를 지정하고 정규 분포를 가진 랜덤한 값으로 입력
  np.random.seed(1)
  P = np.random.normal(scale=1./K, size=(num_users, K))
  Q = np.random.normal(scale=1./K, size=(num_items, K))

  # R > 0인 행 위치, 열 위치, 값을 non_zeros 리스트 객체에 저장
  non_zeros = [(i,j,R[i,j]) for i in range(num_users) for j in range(num_items) if R[i,j] > 0]

  # SGD 기법으로 P와 Q 매트릭스를 계속 업데이트
  for step in range(steps):
    for i, j, r in non_zeros:
      # 실제 값과 예측 값의 차이인 오류 값 구함
      eij = r - np.dot(P[i,:], Q[j,:].T)
      # Regularization을 반영한 SGD 업데이트 공식 적용
      P[i,:] = P[i,:] + learning_rate * (eij * Q[j,:] - r_lambda*P[i,:])
      Q[j,:] = Q[j,:] + learning_rate * (eij * P[i,:] - r_lambda*Q[j,:])

    rmse = get_rmse(R, P, Q, non_zeros)
    if (step % 10) == 0:
      print("### iteration step :", step, " rmse :", rmse)

  return P, Q

In [5]:
import pandas as pd
import numpy as np

movies = pd.read_csv('/content/movies.csv')
ratings = pd.read_csv('/content/ratings.csv')
ratings = ratings[['userId', 'movieId', 'rating']]
ratings_matrix = ratings.pivot_table('rating', index = 'userId', columns = 'movieId')

# title 칼럼을 얻기 위해 movies와 조인 수행
rating_movies = pd.merge(ratings, movies, on = 'movieId')

# columns='title'로 title 칼럼으로 pivot 수행
ratings_matrix = rating_movies.pivot_table('rating', index='userId', columns='title')


In [6]:
P, Q = matrix_factorization(ratings_matrix.values, K=50, steps=200, learning_rate=0.01,
                            r_lambda=0.01)
pred_matrix = np.dot(P, Q.T)

### iteration step : 0  rmse : 2.9023619751337115
### iteration step : 10  rmse : 0.7335768591017939
### iteration step : 20  rmse : 0.5115539026853438
### iteration step : 30  rmse : 0.37261628282537734
### iteration step : 40  rmse : 0.29608182991810145
### iteration step : 50  rmse : 0.2520353192341621
### iteration step : 60  rmse : 0.22487503275269882
### iteration step : 70  rmse : 0.20685455302331512
### iteration step : 80  rmse : 0.19413418783028674
### iteration step : 90  rmse : 0.1847008200272031
### iteration step : 100  rmse : 0.17742927527209082
### iteration step : 110  rmse : 0.17165226964707506
### iteration step : 120  rmse : 0.16695181946871496
### iteration step : 130  rmse : 0.16305292191997453
### iteration step : 140  rmse : 0.159766919296796
### iteration step : 150  rmse : 0.15695986999457337
### iteration step : 160  rmse : 0.15453398186715442
### iteration step : 170  rmse : 0.1524161855107769
### iteration step : 180  rmse : 0.1505508073962834
### iteration

In [7]:
# 예측 사용자-아이템 평점 행렬 => 영화 타이틀을 칼럼명으로 갖는 DataFrame으로 변경
ratings_pred_matrix = pd.DataFrame(data=pred_matrix, index=ratings_matrix.index,
                                   columns=ratings_matrix.columns)
ratings_pred_matrix.head(3)

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,3.055084,4.092018,3.564130,4.502167,3.981215,1.271694,3.603274,2.333266,5.091749,3.972454,...,1.402608,4.208382,3.705957,2.720514,2.787331,3.475076,3.253458,2.161087,4.010495,0.859474
2,3.170119,3.657992,3.308707,4.166521,4.311890,1.275469,4.237972,1.900366,3.392859,3.647421,...,0.973811,3.528264,3.361532,2.672535,2.404456,4.232789,2.911602,1.634576,4.135735,0.725684
3,2.307073,1.658853,1.443538,2.208859,2.229486,0.780760,1.997043,0.924908,2.970700,2.551446,...,0.520354,1.709494,2.281596,1.782833,1.635173,1.323276,2.887580,1.042618,2.293890,0.396941


In [8]:
import pandas as pd
import numpy as np

# 사용자가 아직 보지 않은 영화 목록 뽑기
def get_unseen_movies(ratings_matrix: pd.DataFrame, user_id: int):

    if user_id not in ratings_matrix.index:
        raise KeyError(f"user_id {user_id}가 ratings_matrix.index에 없습니다.")

    # Step 1. 해당 유저의 평점 벡터
    user_ratings = ratings_matrix.loc[user_id]

    # Step 2. 본 영화: 평점이 0보다 크거나(명시적 평점) / NaN이 아닌 값
    seen_movies = user_ratings[user_ratings.fillna(0) > 0].index.tolist()

    # Step 3. 전체 영화 목록에서 본 영화를 제외 → 안 본 영화
    total_movies = ratings_matrix.columns.tolist()
    unseen_list = [m for m in total_movies if m not in seen_movies]
    return unseen_list


# 예측 평점 행렬에서 안 본 영화 중 top-N 추천 뽑기 (Series로 반환)
def recomm_movie_by_userid(pred_matrix: pd.DataFrame, user_id: int,
                           unseen_list: list, top_n: int = 10) -> pd.Series:

    if user_id not in pred_matrix.index:
        raise KeyError(f"user_id {user_id}가 pred_matrix.index에 없습니다.")

    if len(unseen_list) == 0:
        return pd.Series(dtype=float)

    # Step 1. 해당 유저의 예측 평점에서 미시청 영화만 선택
    candidate_cols = [c for c in unseen_list if c in pred_matrix.columns]
    if len(candidate_cols) == 0:
        return pd.Series(dtype=float)

    user_pred = pred_matrix.loc[user_id, candidate_cols]

    # Step 2. NaN 제거
    user_pred = user_pred.dropna()

    # Step 3. 내림차순 정렬 후 top-N
    recomm_series = user_pred.sort_values(ascending=False).head(top_n)
    return recomm_series

In [9]:
## 개인화된 영화 추천
# 사용자가 관람하지 않은 영화명 추출
unseen_list = get_unseen_movies(ratings_matrix, 9)

# 잠재 요인 협업 필터링으로 영화 추천
recomm_movies = recomm_movie_by_userid(ratings_pred_matrix, 9, unseen_list, top_n=10)

# 평점 데이터를 DataFrame으로 생성
recomm_movies = pd.DataFrame(data=recomm_movies.values, index=recomm_movies.index)
columns = ['pred_score']
recomm_movies

,0
title,
Rear Window (1954),5.704612
"South Park: Bigger, Longer and Uncut (1999)",5.451100
Rounders (1998),5.298393
Blade Runner (1982),5.244951
Roger & Me (1989),5.191962
Gattaca (1997),5.183179
Ben-Hur (1959),5.130463
Rosencrantz and Guildenstern Are Dead (1990),5.087375
"Big Lebowski, The (1998)",5.038690


##8

In [2]:
!pip install numpy==1.26.4 scipy==1.11.4 cython==0.29.36

  Using cached numpy-1.26.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.3 MB/s eta 0:00:00
Using cached numpy-1.26.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.0 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.8/35.8 MB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 988.3/988.3 kB 57.6 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.3.4
    Uninstalling numpy-2.3.4:
      Successfully uninstalled numpy-2.3.4
  Attempting uninstall: cython
    Found existing installation: Cython 3.0.12
    Uninstalling Cython-3.0.12:
      Successfully uninstalled Cython-3.0.12
  Attempting uninstall: scipy
    Found existing installation: scipy 1.16.3
    Uninstalling scipy-1.16.3:
      Successfully uninstalled scipy-1.16.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. 

In [2]:
!pip install scikit-surprise

In [3]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

In [4]:
data = Dataset.load_builtin('ml-100k')

# 수행 시마다 동일하게 데이터를 분할하기 위해 random_state 값 부여
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

Dataset ml-100k could not be found. Do you want to download it? [Y/n] Y
Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


In [5]:
algo = SVD(random_state=0)
algo.fit(trainset)

In [6]:
predictions = algo.test(testset)
print('predictions type:', type(predictions), 'size:', len(predictions))
print('prediction 결과의 최초 5개 추출')
predictions[:5]

predictions type: <class 'list'> size: 25000
prediction 결과의 최초 5개 추출


[Prediction(uid='120', iid='282', r_ui=4.0, est=3.5114147666251547, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=3.573872419581491, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=4.033583485472447, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.8463639495936905, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.1807542478219157, details={'was_impossible': False})]

In [7]:
[(pred.uid, pred.iid, pred.est) for pred in predictions[:3]]

[('120', '282', 3.5114147666251547),
 ('882', '291', 3.573872419581491),
 ('535', '507', 4.033583485472447)]

In [8]:
# 사용자 아이디, 아이템 아이디는 문자열로 입력해야 함
uid = str(196)
iid = str(302)
pred = algo.predict(uid, iid)
print(pred)

user: 196        item: 302        r_ui = None   est = 4.49   {'was_impossible': False}


In [9]:
accuracy.rmse(predictions)

RMSE: 0.9467


0.9466860806937948

In [11]:
import pandas as pd

ratings = pd.read_csv('/content/ratings.csv')

# ratings_noh.csv 파일로 언로드 시 인덱스와 헤더를 모두 제거한 새로운 파일 생성
ratings.to_csv('/content/ratings_noh.csv', index=False, header=False)

In [15]:
df = pd.read_csv('/content/ratings_noh.csv')
df = df.dropna(subset=[df.columns[2], df.columns[3]])  # rating, timestamp 열의 NaN 제거
df.to_csv('/content/ratings_noh_clean.csv', index=False)

In [17]:
from surprise import Reader, Dataset

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
data = Dataset.load_from_file('/content/ratings_noh_clean.csv', reader=reader)


In [18]:
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

# 수행 시마다 동일한 결과를 도출하기 위해 random_state 설정
algo = SVD(n_factors=50, random_state=0)

# 학습 데이터 세트로 학습하고 나서 테스트 데이터 세트로 평점 예측 후 RMSE 평가
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8749


0.8748922458806635

In [19]:
import pandas as pd
from surprise import Reader, Dataset

ratings = pd.read_csv('/content/ratings.csv')
reader = Reader(rating_scale=(0.5,5.0))

# ratings DataFrame에서 칼럼은 사용자 아이디, 아이템 아이디, 평점 순서를 지켜야 함
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

algo = SVD(n_factors=50, random_state=0)
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

In [20]:
from surprise.model_selection import cross_validate

# 판다스 DataFrame에서 Surprise 데이터 세트로 데이터 로딩
ratings = pd.read_csv('/content/ratings.csv') # reading data in pandas df
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

algo = SVD(random_state=0)
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8720  0.8674  0.8772  0.8761  0.8739  0.8733  0.0035  
MAE (testset)     0.6742  0.6666  0.6743  0.6701  0.6725  0.6715  0.0029  
Fit time          1.42    1.50    1.45    1.47    1.67    1.50    0.09    
Test time         0.14    0.11    0.16    0.11    0.31    0.17    0.08    


{'test_rmse': array([0.87197775, 0.86740507, 0.87716693, 0.87614484, 0.87393259]),
 'test_mae': array([0.67420217, 0.66660761, 0.67432098, 0.67010121, 0.67250276]),
 'fit_time': (1.419424057006836,
  1.4959492683410645,
  1.4507651329040527,
  1.4653127193450928,
  1.6725184917449951),
 'test_time': (0.13790583610534668,
  0.10926651954650879,
  0.15707063674926758,
  0.11069488525390625,
  0.31368422508239746)}

In [21]:
from surprise.model_selection import GridSearchCV

# 최적화할 파라미터를 딕셔너리 형태로 지정
param_grid = {'n_epochs':[20,40,60], 'n_factors':[50,100,200]}

# cv를 3개 폴드 세트로 지정, 성능 평가는 rmse, mse로 수행하도록 GridSearchCV 구성
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)

# 최고 RMSE Evaluation 점수와 그때의 하이퍼 파라미터
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.8769358587073152
{'n_epochs': 20, 'n_factors': 50}


In [22]:
# 다음 코드는 train_test_split()으로 분리되지 않는 데이터 세트에 fit()을 호출해 오류가 발생
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
algo = SVD(n_factors=50, random_state=0)
algo.fit(data)

AttributeError: 'DatasetAutoFolds' object has no attribute 'n_users'

In [29]:
from surprise.dataset import DatasetAutoFolds

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5,5))
# DatasetAutoFolds 클래스를 ratings_noh.csv 파일 기반으로 생성
data_folds = DatasetAutoFolds(ratings_file='/content/ratings_noh.csv', reader=reader)

# 전체 데이터를 학습 데이터로 생성함
trainset = data_folds.build_full_trainset()

In [31]:
algo = SVD(n_epochs=20, n_factors=50, random_state=0)
algo.fit(trainset)

In [32]:
# 영화에 대한 상세 속성 정보 DataFrame 로딩
movies = pd.read_csv('/content/movies.csv')

# userId=9의 movieId 데이터를 추출해 movieId=42 데이터가 있는지 확인
movieIds = ratings[ratings['userId']==9]['movieId']

if movieIds[movieIds==42].count() == 0:
  print('사용자 아이디 9는 영화 아이디 42의 평점 없음')

print(movies[movies['movieId']==42])

사용자 아이디 9는 영화 아이디 42의 평점 없음
    movieId                   title              genres
38       42  Dead Presidents (1995)  Action|Crime|Drama


In [33]:
uid = str(9)
iid = str(42)

pred = algo.predict(uid, iid, verbose=True)

user: 9          item: 42         r_ui = None   est = 3.05   {'was_impossible': False}


In [34]:
def get_unseen_surprise(ratings, movies, userId):
  # 입력값으로 들어온 userId에 해당하는 사용자가 평점을 매긴 모든 영화를 리스트로 생성
  seen_movies = ratings[ratings['userId']==userId]['movieId'].tolist()

  # 모든 영화의 movieId를 리스트로 생성
  total_movies = movies['movieId'].tolist()

  # 모든 영화의 movieId 중 이미 평점을 매긴 영화의 movieId를 제외한 후 리스트로 생성
  unseen_movies = [movie for movie in total_movies if movie not in seen_movies]
  print('평점 매긴 영화 수:', len(seen_movies),
        '추천 대상 영화 수:', len(unseen_movies),
        '전체 영화 수:', len(total_movies))
  return unseen_movies

unseen_movies = get_unseen_surprise(ratings, movies, 9)

평점 매긴 영화 수: 46 추천 대상 영화 수: 9696 전체 영화 수: 9742


In [35]:

def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n=10):

  # 알고리즘 객체의 predict() 메서드를 평점이 없는 영화에 반복 수행한 후 결과를 list 객체로 저장
  predictions = [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]

  # predictions list 객체는 surprise의 Predictions 객체를 원소로 가지고 있음
  # [Prediction(uid='9', iid='1', est=3.69), Predictions(uid='9', iid='2', est=2.98),,,]

  # 이를 est 값으로 정렬하기 위해서 아래의 sortkey_est 함수를 정의함
  # sortkey_est 함수는 list 객체의 sort() 함수의 키 값으로 사용되어 정렬 수행
  def sortkey_est(pred):
    return pred.est

  # sortkey_est() 반환값의 내림 차순으로 정렬 수행하고 top_n 개의 최상위 값 추출
  predictions.sort(key=sortkey_est, reverse=True)
  top_predictions = predictions[:top_n]

  # top_n으로 추철된 영화의 정보 추출. 영화 아이디, 추천 예상 평점, 제목 추출
  top_movie_ids = [int(pred.iid) for pred in top_predictions]
  top_movie_rating = [pred.est for pred in top_predictions]
  top_movie_titles = movies[movies.movieId.isin(top_movie_ids)]['title']
  top_movie_preds = [(id, title, rating) for id, title, rating in
                     zip(top_movie_ids, top_movie_titles, top_movie_rating)]
  return top_movie_preds

unseen_movies = get_unseen_surprise(ratings, movies, 9)
top_movie_preds = recomm_movie_by_surprise(algo, 9, unseen_movies, top_n=10)

print('#### Top-10 추천 영화 리스트 ####')
for top_movie in top_movie_preds:
  print(top_movie[1], ':', top_movie[2])

평점 매긴 영화 수: 46 추천 대상 영화 수: 9696 전체 영화 수: 9742
#### Top-10 추천 영화 리스트 ####
Hoop Dreams (1994) : 4.0969882778843685
Shawshank Redemption, The (1994) : 4.094067061742648
Schindler's List (1993) : 4.076455576112923
Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964) : 4.075274852141534
One Flew Over the Cuckoo's Nest (1975) : 4.061211964020342
Star Wars: Episode V - The Empire Strikes Back (1980) : 4.060948975712249
Princess Bride, The (1987) : 4.046987766463949
Brazil (1985) : 4.045330923717742
Lawrence of Arabia (1962) : 4.030142176948666
Cool Hand Luke (1967) : 4.021767099275664
